In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
from copy import deepcopy

from pydmd import DMD
from pydmd.bopdmd import BOPDMD
from pydmd.plotter import plot_eigs, plot_summary 

import matplotlib.pyplot as plt

In [ ]:
_ = pd.read_csv('cdii.csv').drop(columns=['Unnamed: 0'])
Xn = _.values

t=np.linspace(0, 230, 24)

In [ ]:
dmd = DMD(svd_rank=24, tlsq_rank=2, exact=True, opt=True)
dmd.fit(Xn)

Aexa = np.linalg.multi_dot([dmd.modes, np.diag(dmd.eigs), np.linalg.pinv(dmd.modes)])

In [ ]:
def hard_threshold(X, gamma=0.01):
    # Hard thresholding for L0 norm.
    X[np.abs(X) ** 2 < 5 * gamma] = 0.0
    return X

def soft_threshold(X, gamma=0.05):
    # Soft thresholding for L1 norm.
    return np.sign(X) * np.maximum(np.abs(X) - gamma, 0.0)

In [ ]:
bopdmd = BOPDMD(
    svd_rank=24,
    num_trials=10,
    trial_size=0.8,
    varpro_opts_dict={"verbose":True, "tol":0.8},
    eig_constraints={"imag"},
    use_proj=False,
    mode_proxy=hard_threshold,
    compute_A=True
)

bopdmd.fit(Xn, t)

In [ ]:
Abop = np.linalg.multi_dot([bopdmd.modes, np.diag(bopdmd.eigs), np.linalg.pinv(bopdmd.modes)])

In [ ]:
rows = np.random.randint(0,1807,10).reshape(10,1)
cols = np.random.randint(0,1807,10).reshape(1,10)

# Plotting A
plt.subplot(1, 2, 1)
plt.imshow(np.abs(Aexa[rows, cols]), cmap='RdBu', vmin=-0.01, vmax=0.01)
plt.title('Matrix A (exact DMD)')
plt.colorbar()

# Plotting Aexa
plt.subplot(1, 2, 2)
plt.imshow(np.abs(bopdmd.A[rows, cols]), cmap='RdBu', vmin=-0.1, vmax=0.1)
plt.title('Matrix A (BOPDMD)')
plt.colorbar()

plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
plt.hist(Aexa.flatten(), bins=np.linspace(-0.2,0.2,50), color='blue', alpha=0.3, label='Aexa', density=True)
plt.hist(Abop.flatten(), bins=np.linspace(-0.2,0.2,50), color='red', alpha=0.3, label='Abop', density=True)
plt.title('Distribution of values in Aexa and Abop')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
rndset = np.random.choice(range(Xn.shape[0]), size=3, replace=False)

plt.figure(figsize=(10, 12))

# Plotting the first 3 rows of Xn as dots
for i, value in enumerate(rndset):
    plt.subplot(3, 1, i+1)
    plt.scatter(t, Xn[value], label=f'Xn Row {value+1}', marker='o')
    plt.plot(t, bopdmd.reconstructed_data[value], label=f'Reconstructed Data (dop) Row {value+1}', linestyle='--', color='red')
    plt.plot(t, dmd.reconstructed_data[value], label=f'Reconstructed Data (exa) Row {value+1}', linestyle='--', color='green')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
reconstruction_mean, reco_variance = bopdmd.forecast(t)

In [ ]:
plt.figure(figsize=(50, 5))
plt.imshow(reco_variance, cmap='RdBu', aspect=0.01, vmax=3)
plt.colorbar()
plt.title('Reconstruction Variance')
plt.show()
